# Crawling Web Portal Berita Antaranews

## Pengertian Crawling

Crawling, dalam konteks mesin pencari, adalah proses otomatis di mana sebuah program (disebut crawler atau spider) menjelajahi internet dari satu halaman web ke halaman web lainnya, mengikuti tautan yang ditemukannya. Bayangkan seperti sebuah robot kecil yang berkeliaran di dunia maya, membaca setiap halaman yang dikunjunginya.

## Proses Crawling

Dalam Kasus ini, kita mencoba untuk membuat crawler untuk mendapatkan data dari penyedia website berita yaitu Antaranews

### Import library

In [12]:
import requests as req
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re

### Scraping url dari halaman utama kategori

---



In [13]:
urls = [
    "https://www.antaranews.com/politik",
    "https://www.antaranews.com/politik/2",
    "https://www.antaranews.com/politik/3",
    "https://www.antaranews.com/politik/4",
    "https://www.antaranews.com/politik/5",
    "https://www.antaranews.com/ekonomi",
    "https://www.antaranews.com/ekonomi/2",
    "https://www.antaranews.com/ekonomi/3",
    "https://www.antaranews.com/ekonomi/4",
    "https://www.antaranews.com/ekonomi/5"
]

articles_url = []

results = []

def get_articles_url(url):
  articles_url = []
  res = req.get(url)
  if res.status_code == 200:
    soup = bs(res.text, "html.parser")

    articles = soup.find("div", class_="wrapper__list__article").find_all("a")

    for i,article in enumerate(articles):
      if re.search("https://www.antaranews.com/berita/*/*", article["href"]):
        articles_url.append(article["href"])
      else : continue
  else : print("Error")

  return list(dict.fromkeys(articles_url))

for url in urls:
  articles_url += get_articles_url(url)

articles_url.pop()
articles_url.pop()

len(articles_url)

142

### Scraping data dari halaman detail berita

In [14]:
for url in articles_url:
  res = req.get(url)
  if res.status_code == 200:
    soup = bs(res.text, "html.parser")

    title = soup.find("h1").text
    content = soup.find("div", class_="wrap__article-detail-content").text
    image = soup.find("div", class_="wrap__article-detail-image").find("img")["src"]
    category = soup.find("ul", class_="breadcrumbs").find_all("a")[1].text

    results.append({
        'title': title,
        'content': content,
        'image': image,
        'url': url,
        'category': category,
        'source': 'antaranews'
    })
  else: break

### Konversi ke dataframe

In [15]:
df = pd.DataFrame(results)

df

,title,content,image,url,category,source
0,Gus Ipul tanggalkan jabatan Wali Kota Pasuruan,"\n""Per hari ini juga saya mundur sebagai Wali ...",https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4324327/gus-...,Politik,antaranews
1,Presiden Jokowi lantik Aida Suwandi jadi Anggo...,"\n""Demi Allah saya bersumpah bahwa saya tidak ...",https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4324243/pres...,Politik,antaranews
2,Presiden Jokowi lantik Eddy Hartono jadi Kepal...,"\n""Demi Allah saya bersumpah bahwa saya akan s...",https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4324219/pres...,Politik,antaranews
3,Wakil KSAD tetapkan 500 warga sipil sebagai ko...,"\n“Dengan mengucap Bismillahirrahmanirrahim, p...",https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4324191/waki...,Politik,antaranews
4,"Relawan Prabowo-Gibran: Gerakan ""tusuk 3 paslo...",\n\t\t\t\t\t\t\t\tJakarta (ANTARA) - Koordinat...,https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4324179/rela...,Politik,antaranews
...,...,...,...,...,...,...
137,OJK: Semua bank miliki sistem deteksi rekening...,\nKami meminta bank untuk meneliti lebih lanju...,https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4322531/ojk-...,Ekonomi,antaranews
138,PT PAL rampungkan “docking refurbishment” KRI ...,\n\t\t\t\t\t\t\t\tSurabaya (ANTARA) - PT PAL I...,https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4322475/pt-p...,Ekonomi,antaranews
139,Rupiah datar di tengah pasar antisipasi data i...,\n\t\t\t\t\t\t\t\tJakarta (ANTARA) - Nilai tuk...,https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4322419/rupi...,Ekonomi,antaranews
140,PGN: Pembayaran tagihan jargas dapat melalui a...,\n\t\t\t\t\t\t\t\tJakarta (ANTARA) - PT PGN Tb...,https://img.antaranews.com/cache/1200x800/2024...,https://www.antaranews.com/berita/4322379/pgn-...,Ekonomi,antaranews


In [16]:
df.to_csv("antaranews.csv", index=False)